## Нахождение минимума функции методом Пиявского

In [105]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from operator import itemgetter
from matplotlib import pyplot as plt
from IPython.display import display
import numpy as np
import math
%matplotlib inline

In [106]:
def pairwise(l):
    for t in zip(l, l[1:]):
        yield t

In [107]:
# n step function for interactive visualization
def pijavskij_n(f, L, a, b, N=3):
    xa, ya, xb, yb = a, f(a), b, f(b)
    x = (yb - ya - L * (xb + xa))/(-2 * L)
    y = ya - L*(x - xa)
    pjvsk = [[(xa, ya), (x, y), (xb, yb)]]
    points = [[x, f(x)]]
    index = 1
    for i in range(N-1):
        r = pjvsk[-1].copy()
        r[index] = points[-1]
        (xa, ya), (xb, yb) = r[index-1], r[index]
        x1 = (yb - ya - L * (xb + xa))/(-2 * L)
        y1 = ya - L*(x1 - xa)
        (xa, ya), (xb, yb) = r[index], r[index+1]
        x2 = (yb - ya - L * (xb + xa))/(-2 * L)
        y2 = ya - L*(x2 - xa)
        r.append([x1, y1]) 
        r.append([x2, y2])
        r.sort(key=itemgetter(0))    
        pjvsk.append(r.copy())
        xx = sorted(r, key=itemgetter(1))[0][0]
        points.append((xx, f(xx)))
        for i,(xxx, _) in enumerate(r):
            if xxx == xx:
                index = i
                break
    return pjvsk, points

In [108]:
# interactive visualizaiton
@interact_manual(n=(1, 20, 1), L='5', b=(0, 100, 0.1), a=(-100, 100, 0.1))
def plot_funct(f='np.sin(x)*x**3', a=-2.2, b=2.4, n=3, L=1):
    x = np.linspace(a, b, 1000)
    plt.figure(figsize=(20, 10))
    y = (lambda x: eval(f))(x)
    miny = min(y)
    L = float(L)
    plt.plot(x, y, label='initial function', ls='--', color='black')
    
    pijavskij_results, pijavskij_points = pijavskij_n(lambda x:eval(f), L, a, b, n)
    
    for i, r in enumerate(pijavskij_results[-3:], 1):
        rn = np.array(r)
        plt.plot(rn[:, 0], rn[:, 1], label='step #{}'.format(n-(3-i)), color='blue',
                 alpha=[0.2,0.4, 1][i-1], 
                 lw=[15, 6, 1][i-1]);
        
    pp = np.array(pijavskij_points)
    plt.scatter(pp[:,0], pp[:,1], color='red', lw=5)
    plt.axhline(y=miny, color='g')
    
    plt.legend()
    plt.show();

interactive(children=(Text(value='np.sin(x)*x**3', description='f'), FloatSlider(value=-2.2, description='a', min=-100.0), FloatSlider(value=2.4, description='b'), IntSlider(value=3, description='n', max=20, min=1), Text(value='5', description='L'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

In [103]:
# same function but until converging
def pijavskij(f, L, a, b, eps=10e-5):
    xa, ya, xb, yb = a, f(a), b, f(b)
    x = (yb - ya - L * (xb + xa))/(-2 * L)
    y = ya - L*(x - xa)
    pjvsk = [[(xa, ya), (x, y), (xb, yb)]]
    last = (x,f(x))
    index = 1
    steps = 0
    while True:
        steps += 1
        r = pjvsk[-1].copy()
        r[index] = last
        (xa, ya), (xb, yb) = r[index-1], r[index]
        x1 = (yb - ya - L * (xb + xa))/(-2 * L)
        y1 = ya - L*(x1 - xa)
        (xa, ya), (xb, yb) = r[index], r[index+1]
        x2 = (yb - ya - L * (xb + xa))/(-2 * L)
        y2 = ya - L*(x2 - xa)
        r.append([x1, y1]) 
        r.append([x2, y2])
        r.sort(key=itemgetter(0))    
        pjvsk.append(r.copy())
        xx = sorted(r, key=itemgetter(1))[0][0]
        for i,(xxx, _) in enumerate(r):
            if xxx == xx:
                index = i
                break
        next = f(xxx)
        if np.isclose(next,last[1],eps):
            print("#steps:", steps)
            return (xx, next)
        last = (xx,next)


In [104]:
pijavskij(lambda x:0.1667*np.exp(-0.8571*x)-x, L= np.exp(2)+2, a=-2.2, b=2.4)

#steps: 58


(2.3999454672389455, -2.3786349113885668)